In [ ]:
import numpy as np

# Your class name mapping, ordered by class index
CLASS_NAMES = [
    '2-onder-1-kapwoning',
    'Bovenwoning/Benedenwoning/Maisonette',
    'Corridorflat/Galerijflat',
    'Geschakelde 2-onder-1-kapwoning',
    'Geschakelde woning',
    'Halfvrijstaande woning',
    'Hoekwoning/Eindwoning',
    'Portiekflat/Portiekwoning',
    'Tussenwoning',
    'Vrijstaande woning'
]

def classify_with_threshold(model, input_data, threshold=0.5, is_torch=False):
    """
    Universal classification function that works for sklearn, Keras, PyTorch.

    Parameters:
        model: The trained model
        input_data: Input samples (NumPy for sklearn/Keras, torch.Tensor for PyTorch)
        threshold: Minimum probability threshold
        is_torch: Set to True if using PyTorch

    Returns:
        List of (class name or 'Unknown', probability)
    """
    if is_torch:
        import torch.nn.functional as F
        model.eval()
        with torch.no_grad():
            logits = model(input_data)
            probs = F.softmax(logits, dim=1).cpu().numpy()
    else:
        try:
            probs = model.predict_proba(input_data)
        except AttributeError:
            probs = model.predict(input_data)

    results = []
    for prob in probs:
        max_prob = np.max(prob)
        predicted_index = np.argmax(prob)
        predicted_class = CLASS_NAMES[predicted_index] if max_prob >= threshold else "Unknown"
        results.append((predicted_class, max_prob))

    return results


# For RFC
rfc_results = classify_with_threshold(rfc_model, rfc_input_data)

# For CNN (e.g., Keras)
# cnn_results = classify_with_threshold(cnn_model, cnn_input_data)

# For MLP
# mlp_results = classify_with_threshold(mlp_model, mlp_input_data)

# For multimodal model
#multi_results = classify_with_threshold(multi_model, combined_input)

# Print predictions
for i, (label, prob) in enumerate(rfc_results):
    print(f"Sample {i}: {label} with probability {prob:.2f}")

#for pytorch model
def classify_with_threshold_pytorch(model, dataloader, class_names, threshold=0.5):
    model.eval()
    results = []

    with torch.no_grad():
        for x_batch, _ in dataloader:
            x_batch = x_batch.to(device)
            logits = model(x_batch)
            probs = F.softmax(logits, dim=1).cpu().numpy()

            for prob in probs:
                max_prob = np.max(prob)
                predicted_index = np.argmax(prob)
                predicted_class = class_names[predicted_index] if max_prob >= threshold else "Unknown"
                results.append((predicted_class, max_prob))

    return results

#mlp_predictions = classify_with_threshold_pytorch(model, test_loader, CLASS_NAMES, threshold=0.5)

# Print a few results
#for i, (label, prob) in enumerate(mlp_predictions[:10]):
#    print(f"Sample {i}: {label} with probability {prob:.2f}")

#for the multi modal
def classify_with_threshold_multimodal(model, dataloader, class_names, device, threshold=0.5):
    """
    Runs inference on a multimodal model with image and tabular inputs, returning predicted class names
    or 'Unknown' if confidence is below the threshold.

    Parameters:
        model: Trained PyTorch model
        dataloader: DataLoader providing (images, tabular_data, labels)
        class_names: List of class names (ordered by label index)
        device: 'cuda' or 'cpu'
        threshold: Minimum probability required to return a class name

    Returns:
        List of (predicted class name or 'Unknown', probability)
    """
    model.eval()
    results = []

    with torch.no_grad():
        for images, tabular_data, _ in dataloader:
            images = images.to(device)
            tabular_data = tabular_data.to(device)

            outputs = model(images, tabular_data)
            probs = F.softmax(outputs, dim=1).cpu().numpy()

            for prob in probs:
                max_prob = np.max(prob)
                pred_idx = np.argmax(prob)
                class_name = class_names[pred_idx] if max_prob >= threshold else "Unknown"
                results.append((class_name, max_prob))

    return results

results = classify_with_threshold_multimodal(model, test_loader, CLASS_NAMES, device=device, threshold=0.5)

# Print the first few results
for i, (label, prob) in enumerate(results[:10]):
    print(f"Sample {i}: {label} with confidence {prob:.2f}")
